In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import joblib

In [2]:
df=pd.read_csv("C:/Users/priya/Desktop/Major project/Multiple Disease Prediction System/Datasets/kidney_disease.csv")
df

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51,7200,5.9,no,no,no,good,no,no,notckd


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [4]:
for col in df.select_dtypes(include='object').columns:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace(['?', '\t?', 'na', 'nan', ''], np.nan)
    )


In [5]:
for col in ["pcv","wc","rc"]:
    df[col]=pd.to_numeric(df[col],errors='coerce')

In [6]:
numeric_cols=df.select_dtypes(include=['float64','int64']).columns
df[numeric_cols]=df[numeric_cols].fillna(df[numeric_cols].mean())

In [7]:
categorical_cols=df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df[col].mode().size > 0:
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna("unknown")


In [8]:
encoder={}
for col in categorical_cols:
    le=LabelEncoder()
    df[col]=le.fit_transform(df[col])
    encoder[col]=le

In [9]:
X=df.drop("classification",axis=1)
y=df["classification"]

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
print(X_train)

      id   age     bp        sg        al        su  rbc  pc  pcc  ba  ...  \
157  157  62.0   70.0  1.025000  3.000000  0.000000    1   0    0   0  ...   
109  109  54.0   70.0  1.017408  1.016949  0.450142    1   1    0   0  ...   
17    17  47.0   80.0  1.017408  1.016949  0.450142    1   1    0   0  ...   
347  347  43.0   60.0  1.025000  0.000000  0.000000    1   1    0   0  ...   
24    24  42.0  100.0  1.015000  4.000000  0.000000    1   0    0   1  ...   
..   ...   ...    ...       ...       ...       ...  ...  ..  ...  ..  ...   
71    71  46.0   60.0  1.010000  1.000000  0.000000    1   1    0   0  ...   
106  106  50.0   90.0  1.017408  1.016949  0.450142    1   1    0   0  ...   
270  270  23.0   80.0  1.025000  0.000000  0.000000    1   1    0   0  ...   
348  348  38.0   80.0  1.020000  0.000000  0.000000    1   1    0   0  ...   
102  102  17.0   60.0  1.010000  0.000000  0.000000    1   1    0   0  ...   

     hemo        pcv            wc        rc  htn  dm  cad  app

In [11]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [12]:
nb_model=GaussianNB()
nb_model.fit(X_train_scaled,y_train)

GaussianNB()

In [13]:
y_pred_nb=nb_model.predict(X_test_scaled)
y_pred_nb

array([1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1])

In [14]:
dt_model = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,
    random_state=42
)
dt_model.fit(X_train_scaled, y_train)

DecisionTreeClassifier(random_state=42)

In [15]:
y_pred_dt = dt_model.predict(X_test_scaled)
y_pred_dt

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1])

In [16]:
print("Evaluate Model: Naive Bayes")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_nb))
print("Classification Report:\n", classification_report(y_test, y_pred_nb))

Evaluate Model: Naive Bayes
Accuracy: 0.95
Confusion Matrix:
 [[70  6]
 [ 0 44]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96        76
           1       0.88      1.00      0.94        44

    accuracy                           0.95       120
   macro avg       0.94      0.96      0.95       120
weighted avg       0.96      0.95      0.95       120



In [17]:
print("Evaluate Model: Decision Tree")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_dt))

Evaluate Model: Decision Tree
Accuracy: 0.9916666666666667
Confusion Matrix:
 [[76  0]
 [ 1 43]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99        76
           1       1.00      0.98      0.99        44

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



In [18]:
joblib.dump(nb_model, "kidney_naive_bayes.pkl")
joblib.dump(dt_model, "kidney_decision_tree.pkl")
joblib.dump(scaler, "kidney_scaler.pkl")
joblib.dump(encoder, "kidney_label_encoders.pkl")


['kidney_label_encoders.pkl']